# Downloading IBGE data

In [1]:
import os
import zipfile
from ftplib import FTP
import pandas as pd
from tqdm import tqdm
import geopandas as gpd
from shapely.geometry import Point
from geopandas.tools import sjoin

# Agregados por Setores Censitários

In [9]:
ftp = FTP("ftp.ibge.gov.br")
ftp.login()
ftp.cwd("Censos/Censo_Demografico_2010/Resultados_do_Universo/Agregados_por_Setores_Censitarios/")
files_on_ftp_path = ftp.nlst(".")

zip_files_to_download = [filename for filename in files_on_ftp_path if ".zip" in filename and "Doc" not in filename]
zip_files_to_download

['AC_20171016.zip',
 'AL_20171016.zip',
 'AM_20171016.zip',
 'AP_20171016.zip',
 'BA_20171016.zip',
 'CE_20171016.zip',
 'DF_20171016.zip',
 'ES_20171016.zip',
 'GO_20171016.zip',
 'MA_20171016.zip',
 'MG_20171016.zip',
 'MS_20171016.zip',
 'MT_20171016.zip',
 'PA_20171016.zip',
 'PB_20171016.zip',
 'PE_20200219.zip',
 'PI_20171016.zip',
 'PR_20171016.zip',
 'RJ_20171016.zip',
 'RN_20171016.zip',
 'RO_20171016.zip',
 'RR_20171016.zip',
 'RS_20171016.zip',
 'SC_20171016.zip',
 'SE_20171016.zip',
 'SP_Capital_20190823.zip',
 'SP_Exceto_a_Capital_20190207.zip',
 'TO_20171016.zip']

In [27]:
for filename in zip_files_to_download:
    print("Downloading {}...".format(filename))
    with open(f"../data/agregados/{filename}", "wb") as fp:
        ftp.retrbinary("RETR {}".format(filename), fp.write)

# SHP files dos setores censitários

In [22]:
ftp = FTP("geoftp.ibge.gov.br")
ftp.login()
ftp.cwd("organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2010/setores_censitarios_shp/")
folders_on_ftp_path = ftp.nlst(".")

uf_folders = [filename for filename in folders_on_ftp_path if len(filename) == 2]

In [23]:
uf_folders

['ac',
 'al',
 'am',
 'ap',
 'ba',
 'ce',
 'df',
 'es',
 'go',
 'ma',
 'mg',
 'ms',
 'mt',
 'pa',
 'pb',
 'pe',
 'pi',
 'pr',
 'rj',
 'rn',
 'ro',
 'rr',
 'rs',
 'sc',
 'se',
 'sp',
 'to']

In [28]:
for uf in uf_folders:
    ftp = FTP("geoftp.ibge.gov.br")
    ftp.login()
    ftp.cwd(f"organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2010/setores_censitarios_shp/{uf}/")
    # the following is needed because of a typo on GO file
    files_on_ftp_path = ftp.nlst(".")
    file_to_download = [filename for filename in files_on_ftp_path if "setores" in filename][0]
    print(f"Downloading {file_to_download} ...")
    with open(f"../data/setores_censitarios/{uf}_setores_censitarios.zip", "wb") as fp:
        ftp.retrbinary(f"RETR {file_to_download}", fp.write)

# Extraindo os zipfiles para uma pasta comum

## SHP

In [6]:
agregados_files = os.listdir("../data/agregados/")
agregados_path_files = [os.path.join("../data/agregados/", file) for file in agregados_files]
agregados_path_files[:3]

['../data/agregados/AM_20171016.zip',
 '../data/agregados/RN_20171016.zip',
 '../data/agregados/PR_20171016.zip']

In [9]:
directory_to_extract_to = "../data/agregados/extracted/"

for path_to_zip_file in tqdm(agregados_path_files):
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

100%|██████████| 28/28 [01:04<00:00,  2.30s/it]


# Setores Censitários

In [16]:
sc_files = [file for file in os.listdir("../data/setores_censitarios/") if ".zip" in file]
sc_path_files = [os.path.join("../data/setores_censitarios/", file) for file in sc_files]
sc_path_files[:3]

['../data/setores_censitarios/am_setores_censitarios.zip',
 '../data/setores_censitarios/ms_setores_censitarios.zip',
 '../data/setores_censitarios/es_setores_censitarios.zip']

In [17]:
directory_to_extract_to = "../data/setores_censitarios/extracted/"

for path_to_zip_file in tqdm(sc_path_files):
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

100%|██████████| 27/27 [00:05<00:00,  4.90it/s]


# Open Addresses Brazil

Source: [https://www.kaggle.com/openaddresses/openaddresses-south-america?select=brazil.csv](https://www.kaggle.com/openaddresses/openaddresses-south-america?select=brazil.csv)

In [17]:
cep_coords = pd.read_csv("../data/open_addresses_south_america/brazil.csv", usecols=["POSTCODE", "LAT", "LON"])
cep_coords.head()

,LON,LAT,POSTCODE
0,-67.0496,-10.071417,69945-000
1,-67.0496,-10.071417,69945-000
2,-67.0496,-10.071417,69945-000
3,-67.0496,-10.071417,69945-000
4,-67.0496,-10.071417,69945-000


In [21]:
cep_coords = cep_coords.groupby(by="POSTCODE").median().reset_index()

In [23]:
cep_coords.shape

(429972, 3)

In [25]:
cep_coords.head()

,POSTCODE,LON,LAT
0,00000-000,-46.663830,-22.257613
1,00000-001,-67.838738,-9.921660
2,00000-002,-67.880508,-9.882072
3,00000-003,-46.514443,-21.820570
4,00000-004,-67.893353,-9.882038


In [58]:
cep_coords_gdf = gpd.GeoDataFrame(
    cep_coords, geometry=gpd.points_from_xy(cep_coords["LON"], cep_coords["LAT"]))

In [59]:
cep_coords_gdf

,POSTCODE,LON,LAT,geometry
0,00000-000,-46.663830,-22.257613,POINT (-46.66383 -22.25761)
1,00000-001,-67.838738,-9.921660,POINT (-67.83874 -9.92166)
2,00000-002,-67.880508,-9.882072,POINT (-67.88051 -9.88207)
3,00000-003,-46.514443,-21.820570,POINT (-46.51444 -21.82057)
4,00000-004,-67.893353,-9.882038,POINT (-67.89335 -9.88204)
...,...,...,...,...
429967,99990-999,-48.562378,-25.574579,POINT (-48.56238 -25.57458)
429968,99999-990,-48.563207,-25.567153,POINT (-48.56321 -25.56715)
429969,99999-998,-41.092554,-20.853443,POINT (-41.09255 -20.85344)
429970,99999-999,-43.998336,-22.912780,POINT (-43.99834 -22.91278)


In [65]:
shp_files = [file for file in os.listdir('../data/setores_censitarios/extracted/') if ".shp" in file]
shp_files[:3]

['24SEE250GC_SIR.shp', '32SEE250GC_SIR.shp', '16SEE250GC_SIR.shp']

In [66]:
gdfs = [gpd.read_file(f'../data/setores_censitarios/extracted/{filename}') for filename in shp_files]
census_sector_gpd = pd.concat(gdfs)
del gdfs

In [67]:
census_sector_gpd = census_sector_gpd[["CD_GEOCODI", "geometry"]]
census_sector_gpd.head()

,CD_GEOCODI,geometry
0,240020805000006,"POLYGON ((-36.90843 -5.58145, -36.90834 -5.582..."
1,240020805000007,"POLYGON ((-36.90586 -5.58354, -36.90556 -5.585..."
2,240020805000008,"POLYGON ((-36.90780 -5.58389, -36.90739 -5.586..."
3,240020805000009,"POLYGON ((-36.90939 -5.58418, -36.90933 -5.585..."
4,240020805000010,"POLYGON ((-36.91135 -5.58195, -36.91135 -5.581..."


In [71]:
pointInPolys = sjoin(cep_coords_gdf, census_sector_gpd, how='left')

/opt/miniconda3/envs/ibge_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4674

  """Entry point for launching an IPython kernel.


In [72]:
pointInPolys.isnull().sum()

POSTCODE        0
LON             0
LAT             0
geometry        0
index_right    15
CD_GEOCODI     15
dtype: int64

In [73]:
pointInPolys.dropna()

,POSTCODE,LON,LAT,geometry,index_right,CD_GEOCODI
0,00000-000,-46.663830,-22.257613,POINT (-46.66383 -22.25761),16996.0,313490505000032
1,00000-001,-67.838738,-9.921660,POINT (-67.83874 -9.92166),688.0,120040105000325
2,00000-002,-67.880508,-9.882072,POINT (-67.88051 -9.88207),688.0,120040105000325
3,00000-003,-46.514443,-21.820570,POINT (-46.51444 -21.82057),24568.0,315180005000240
4,00000-004,-67.893353,-9.882038,POINT (-67.89335 -9.88204),688.0,120040105000325
...,...,...,...,...,...,...
429967,99990-999,-48.562378,-25.574579,POINT (-48.56238 -25.57458),12827.0,411820405000166
429968,99999-990,-48.563207,-25.567153,POINT (-48.56321 -25.56715),12766.0,411820405000105
429969,99999-998,-41.092554,-20.853443,POINT (-41.09255 -20.85344),953.0,320120905000204
429970,99999-999,-43.998336,-22.912780,POINT (-43.99834 -22.91278),7481.0,330260120000035


In [74]:
final_df = pointInPolys.drop(columns=["geometry", "index_right"])

In [77]:
final_df = pd.DataFrame(final_df)

In [79]:
final_df.isnull().sum()

POSTCODE       0
LON            0
LAT            0
CD_GEOCODI    15
dtype: int64

In [80]:
final_df = final_df.dropna()

In [83]:
final_df["CD_GEOCODI"] = final_df["CD_GEOCODI"].astype(int)

/opt/miniconda3/envs/ibge_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [84]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 429994 entries, 0 to 429971
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   POSTCODE    429994 non-null  object 
 1   LON         429994 non-null  float64
 2   LAT         429994 non-null  float64
 3   CD_GEOCODI  429994 non-null  int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 16.4+ MB


In [88]:
# final_df.to_csv("../data/processed/census_code_cep_coordinates.csv", index=False)

In [87]:
final_df

,POSTCODE,LON,LAT,CD_GEOCODI
0,00000-000,-46.663830,-22.257613,313490505000032
1,00000-001,-67.838738,-9.921660,120040105000325
2,00000-002,-67.880508,-9.882072,120040105000325
3,00000-003,-46.514443,-21.820570,315180005000240
4,00000-004,-67.893353,-9.882038,120040105000325
...,...,...,...,...
429967,99990-999,-48.562378,-25.574579,411820405000166
429968,99999-990,-48.563207,-25.567153,411820405000105
429969,99999-998,-41.092554,-20.853443,320120905000204
429970,99999-999,-43.998336,-22.912780,330260120000035


In [2]:
census_code_cep_coordinates = pd.read_csv("../data/processed/census_code_cep_coordinates.csv")
census_code_cep_coordinates.head()

,POSTCODE,LON,LAT,CD_GEOCODI
0,00000-000,-46.663830,-22.257613,313490505000032
1,00000-001,-67.838738,-9.921660,120040105000325
2,00000-002,-67.880508,-9.882072,120040105000325
3,00000-003,-46.514443,-21.820570,315180005000240
4,00000-004,-67.893353,-9.882038,120040105000325


In [24]:
pessoa_renda_paths = []
for dirname, dirnames, filenames in os.walk('../data/agregados/extracted/'):
    # get all pessoarenda paths
    for filename in filenames:
        if ("PessoaRenda" in filename) and (".csv" in filename):
            pessoa_renda_paths.append(os.path.join(dirname, filename))

In [25]:
pessoa_renda_paths

['../data/agregados/extracted/SE/Base informaçoes setores2010 universo SE/CSV/PessoaRenda_SE.csv',
 '../data/agregados/extracted/PE_20171016/PE/Base informaçoes setores2010 universo PE/CSV/PessoaRenda_PE.csv',
 '../data/agregados/extracted/SC/Base informaçoes setores2010 universo SC/CSV/PessoaRenda_SC.csv',
 '../data/agregados/extracted/PB/Base informaçoes setores2010 universo PB/CSV/PessoaRenda_PB.csv',
 '../data/agregados/extracted/Base informaçoes setores2010 universo ES/CSV/PessoaRenda_ES.csv',
 '../data/agregados/extracted/MS/Base informaçoes setores2010 universo MS/CSV/PessoaRenda_MS.csv',
 '../data/agregados/extracted/MT/Base informaçoes setores2010 universo MT/CSV/PessoaRenda_MT.csv',
 '../data/agregados/extracted/GO/Base informaçoes setores2010 universo GO/CSV/PessoaRenda_GO.csv',
 '../data/agregados/extracted/AC/Base informaçoes setores2010 universo AC/CSV/PessoaRenda_AC.csv',
 '../data/agregados/extracted/AM/Base informaçoes setores2010 universo AM/CSV/PessoaRenda_AM.csv',
 

# Da documentação de arquivos PessoaRenda_UF

- V020: Pessoas de 10 anos ou mais de idade com ou sem rendimento
- V022: Total do rendimento nominal mensal das pessoas de 10 anos ou mais de idade

In [52]:
dfs = [pd.read_csv(f"{file_path}", sep=";", usecols=["Cod_setor", "V020", "V022"]) for file_path in pessoa_renda_paths]
pessoa_renda_df = pd.concat(dfs)
del dfs

/opt/miniconda3/envs/ibge_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (21,23) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [53]:
pessoa_renda_df

,Cod_setor,V020,V022
0,280010005000001,727,263997
1,280010005000002,378,178925
2,280010005000003,395,91417
3,280010005000004,296,62218
4,280010005000005,51,11121
...,...,...,...
22327,432380425000007,58,97468
22328,432380425000008,223,172340
22329,432380425000009,9,14510
22330,432380425000010,68,79452


In [57]:
# cleaning wrong data
pessoa_renda_df = pessoa_renda_df[pessoa_renda_df["V020"]!="X"]

In [59]:
pessoa_renda_df["V020"] = pessoa_renda_df["V020"].astype(int)
pessoa_renda_df["V022"] = pessoa_renda_df["V022"].astype(int)

/opt/miniconda3/envs/ibge_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/miniconda3/envs/ibge_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [60]:
pessoa_renda_df

,Cod_setor,V020,V022
0,280010005000001,727,263997
1,280010005000002,378,178925
2,280010005000003,395,91417
3,280010005000004,296,62218
4,280010005000005,51,11121
...,...,...,...
22327,432380425000007,58,97468
22328,432380425000008,223,172340
22329,432380425000009,9,14510
22330,432380425000010,68,79452


In [61]:
pessoa_renda_df["renda_per_capita"] = pessoa_renda_df["V022"] / pessoa_renda_df["V020"]

/opt/miniconda3/envs/ibge_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [75]:
pessoa_renda_df = pessoa_renda_df[["Cod_setor", "renda_per_capita"]]

In [77]:
pessoa_renda_df = pessoa_renda_df.rename(columns={
    "Cod_setor": "CD_GEOCODI"
})

In [80]:
cep_coordinates_per_capita_income = census_code_cep_coordinates.merge(pessoa_renda_df, on="CD_GEOCODI")

In [83]:
cep_coordinates_per_capita_income.to_csv("../data/processed/cep_coordinates_per_capita_income.csv", index=False)